# 3D sz's MODEL

- configuration A: (128 x 128 x 64 nodes) 128 CPUS
- configuration B: (256 x 256 x 96 nodes) 256 CPUS
The user should remove visualisation from the python script before running the model on raijin.
In [2]:


In [1]:
import matplotlib.pyplot as plt
import UWGeodynamics as GEO
u = GEO.UnitRegistry # u. +tab to see available units or u = GEO.u
import glucifer

loaded rc file /opt/UWGeodynamics/UWGeodynamics/uwgeo-data/uwgeodynamicsrc


In [2]:
##### Characteristic values of the system

half_rate = 1.8 * u.centimeter / u.year # What is it?
model_length = 4000e3 * u.meter
model_width = 2000e3 * u.meter
surfaceTemp = 273.15 * u.degK
baseModelTemp = 3570 * u.degK
bodyforce = 3300 * u.kilogram / u.metre**3 * 9.81 * u.meter / u.second**2 # ckeck publi schellart

KL = model_length # Characteristic length
Kt = KL / half_rate # Characteristic time 
KM = bodyforce * KL**2 * Kt**2 # Characteristic mass
KT = (baseModelTemp - surfaceTemp) # Characteristic temperature

GEO.scaling_coefficients["[length]"] = KL
GEO.scaling_coefficients["[time]"] = Kt
GEO.scaling_coefficients["[mass]"]= KM
GEO.scaling_coefficients["[temperature]"] = KT


In [3]:
## definition of central object: define the geometry of the problem: container for the field variables

Model = GEO.Model(elementRes=(32, 32, 16), 
                  minCoord=(0. * u.kilometer, -0. * u.kilometer, -660. * u.kilometer), 
                  maxCoord=(4000. * u.kilometer, 2000. * u.kilometer, 20. * u.kilometer), 
                  gravity=(0.0 ,0.0, -9.81 * u.meter / u.second**2))

# Fig = glucifer.Figure()
# Fig.Surface(Model.mesh, Model.projMaterialField, cullface=False, opacity=1)
# viewer = Fig.viewer(resolution=(600,300))
# viewer.window()


## Output

In [4]:
Model.outputDir="3D_sz_ocean_si_faccenda"

## Define Materials
The model has initially 4 materials (air, crust, mantle lithosphere and mantle). We add a fifth material for the sediment. Sediment will only appear if surface processes are turned on...(and if there is sedimentation of course)

In [5]:
# ## Definition of materials

# air               = Model.add_material(name="Air", shape=GEO.shapes.Layer3D(top=Model.top, bottom=2.0 * u.kilometer))
# stickyAir         = Model.add_material(name="StickyAir", shape=GEO.shapes.Layer3D(top=air.bottom, bottom= 0.0 * u.kilometer)) # or water
# basalticrust        = Model.add_material(name="basalticrust", shape=GEO.shapes.Layer3D(top=stickyAir.bottom, bottom=-15.0 * u.kilometer))
# lithosphere          = Model.add_material(name="lithosphere", shape=GEO.shapes.Layer3D(top=basalticrust.bottom * u.kilometer, bottom=-120.0 * u.kilometer))
# asthenosphere = Model.add_material(name="asthenosphere", shape=GEO.shapes.Layer3D(top=lithosphere.bottom * u.kilometer, bottom=-660.0 * u.kilometer))
# mantle            = Model.add_material(name="Mantle", shape=GEO.shapes.Layer3D(top=asthenosphere.bottom, bottom=Model.bottom))
# #sediment          = Model.add_material(name="Sediment")


# ## Definition of slab shape with 1 layer and 1 halfspace: WEAKZONE

# wz_basalticrust = GEO.shapes.Layer3D(top=-0.*u.kilometer, bottom=-15.*u.kilometer)
# slopewz = GEO.shapes.HalfSpace(normal=(-1.,0.,1.), origin=(1995.*u.kilometer,1000.*u.kilometer,-0.*u.kilometer))
# CompositeShape_wz = slopewz & wz_basalticrust 
# wz_basalticrust = Model.add_material(name="wz_basalticcrust", shape=CompositeShape_wz)

# wz_lithosphere = GEO.shapes.Layer3D(top=-15.*u.kilometer, bottom=-120.*u.kilometer)
# #slopeSlab_lithosphere = GEO.shapes.HalfSpace(normal=(-1.,0.,1.), origin=(5000.*u.kilometer,1000.*u.kilometer,-2000.*u.kilometer))
# CompositeShape_lithowz = slopewz & wz_lithosphere 
# wz_lithosphere = Model.add_material(name="wz_lithosphere", shape=CompositeShape_lithowz)



# ## Definition of slab shape with 1 layer and 1 halfspace on weak zone

# slabLayer_basalticrust = GEO.shapes.Layer3D(top=-0.*u.kilometer, bottom=-15.*u.kilometer)
# slopeSlab = GEO.shapes.HalfSpace(normal=(-1.,0.,1.), origin=(2000.*u.kilometer,1000.*u.kilometer,-0.*u.kilometer))
# CompositeShape_bc = slopeSlab & slabLayer_basalticrust 
# Slab_basalticrust = Model.add_material(name="Slab_basalticcrust", shape=CompositeShape_bc)

# slabLayer_lithosphere = GEO.shapes.Layer3D(top=-15.*u.kilometer, bottom=-120.*u.kilometer)
# #slopeSlab_lithosphere = GEO.shapes.HalfSpace(normal=(-1.,0.,1.), origin=(5000.*u.kilometer,1000.*u.kilometer,-2000.*u.kilometer))
# CompositeShape_litho = slopeSlab & slabLayer_lithosphere 
# Slab_lithosphere = Model.add_material(name="Slab_lithosphere", shape=CompositeShape_litho)




# Fig = glucifer.Figure()
# Fig.Surface(Model.mesh, Model.projMaterialField, cullface=False, opacity=0.70)
# viewer = Fig.viewer(resolution=(1200,600))
# viewer = Fig.viewer(axis=True)
# viewer.window()

# # Model_MaterialField=(Model.projMaterialField)


In [14]:
## Definition of materials

Model.materialField.data[:] = 0
Model.materials = []

air               = Model.add_material(name="Air", shape=GEO.shapes.Layer3D(top=Model.top, bottom=2.0 * u.kilometer))
stickyAir         = Model.add_material(name="StickyAir", shape=GEO.shapes.Layer3D(top=air.bottom, bottom= 0.0 * u.kilometer)) # or water
basalticrust        = Model.add_material(name="basalticrust", shape=GEO.shapes.Layer3D(top=stickyAir.bottom, bottom=-120.0 * u.kilometer))
#lithosphere          = Model.add_material(name="lithosphere", shape=GEO.shapes.Layer3D(top=basalticrust.bottom * u.kilometer, bottom=-120.0 * u.kilometer))
asthenosphere = Model.add_material(name="asthenosphere", shape=GEO.shapes.Layer3D(top=-120.0 * u.kilometer * u.kilometer, bottom=Model.bottom))
#mantle            = Model.add_material(name="Mantle", shape=GEO.shapes.Layer3D(top=asthenosphere.bottom, bottom=Model.bottom))
#sediment          = Model.add_material(name="Sediment")


## Definition of slab shape with 1 layer and 1 halfspace: WEAKZONE

boxwall = GEO.shapes.Box(top=stickyAir.bottom, bottom=-300.0*u.kilometer, minX = 2000.*u.kilometer, minY = 0.*u.kilometer, maxX = 4000.*u.kilometer, maxY = 2000.*u.kilometer)

slableft = GEO.shapes.HalfSpace(normal=(-1.,0.,0.), origin=(1500.*u.kilometer,1000.*u.kilometer,-0.*u.kilometer))
slabright = GEO.shapes.HalfSpace(normal=(1.,0.,0.), origin=(2000.*u.kilometer,1000.*u.kilometer,-0.*u.kilometer))
slopetop = GEO.shapes.HalfSpace(normal=(-.3,0.,1.0), origin=(2000.*u.kilometer,1000.*u.kilometer,-0.*u.kilometer))
slopebottom = GEO.shapes.HalfSpace(normal=(.3,0.,-1.0), origin=(2000.*u.kilometer,1000.*u.kilometer,-300.*u.kilometer))
top = GEO.shapes.HalfSpace(normal=(-0.,0.,1.), origin=(2000.*u.kilometer,1000.*u.kilometer,-0.*u.kilometer))



CompositeShape_wz =slableft & slopetop & slopebottom & slabright
CompositeShape_final = GEO.shapes.MultiShape([CompositeShape_wz, boxwall])
wz_basalticrust = Model.add_material(name="wz_basalticcrust", shape=CompositeShape_final)






Fig = glucifer.Figure()
Fig.Points(Model.swarm, Model.materialField, cullface=False, opacity=1.)
viewer = Fig.viewer(resolution=(1200,600))
viewer = Fig.viewer(axis=True)
viewer.window()

# Model_MaterialField=(Model.projMaterialField)



<IPython.core.display.Javascript object>

In [38]:
boxwall = GEO.shapes.Box(top=stickyAir.bottom, bottom=-300.0*u.kilometer, minX = 2000.*u.kilometer, minY = 0.*u.kilometer, maxX = 4000.*u.kilometer, maxY = 2000.*u.kilometer)

slableft = GEO.shapes.HalfSpace(normal=(-1.,0.,0.), origin=(1500.*u.kilometer,1000.*u.kilometer,-0.*u.kilometer))
slabright = GEO.shapes.HalfSpace(normal=(1.,0.,0.), origin=(2000.*u.kilometer,1000.*u.kilometer,-0.*u.kilometer))
slopetop = GEO.shapes.HalfSpace(normal=(-1.,0.,2.0), origin=(2000.*u.kilometer,1000.*u.kilometer,-0.*u.kilometer))
slopebottom = GEO.shapes.HalfSpace(normal=(1.,0.,-2.0), origin=(2000.*u.kilometer,1000.*u.kilometer,-300.*u.kilometer))
top = GEO.shapes.HalfSpace(normal=(-0.,0.,1.), origin=(2000.*u.kilometer,1000.*u.kilometer,-0.*u.kilometer))



CompositeShape_wz =slableft & slopetop & slopebottom & slabright
CompositeShape_final = GEO.shapes.MultiShape([CompositeShape_wz, boxwall])
wz_basalticrust = Model.add_material(name="wz_basalticcrust", shape=CompositeShape_final)



## Limiters


In [7]:
Model.stressLimiter = 300.0 * u.megapascal
Model.maxViscosity = 5.0e23 * u.pascal * u.second
Model.minViscosity = 1.0e19 * u.pascal * u.second
Model.diffusivity = 1.0e-6 * u.metre**2 / u.second 
Model.capacity    = 1000. * u.joule / (u.kelvin * u.kilogram)

### Material specific definitions


In [8]:
# Diffusivity

air.diffusivity = 1.0e-6 * u.metre**2 / u.second
stickyAir.diffusivity = 1.0e-6 * u.metre**2 / u.second

wz_basalticrust.diffusivity   = 1.1e-6 * u.metre**2 / u.second
wz_lithosphere.diffusivity   = 1.1e-6 * u.metre**2 / u.second

basalticrust.diffusivity = 1.1e-6 * u.metre**2 / u.second
Slab_basalticrust.diffusivity   = 1.1e-6 * u.metre**2 / u.second
lithosphere.diffusivity = 1.1e-6 * u.metre**2 / u.second
Slab_lithosphere.diffusivity = 1.1e-6 * u.metre**2 / u.second
asthenosphere.diffusivity = 1.1e-6 * u.metre**2 / u.second
mantle.diffusivity = 1.1e-6 * u.metre**2 / u.second

Model.diffusivity = 1.0e-6 * u.metre**2 / u.second 


# capacity
air.capacity = 100. * u.joule / (u.kelvin * u.kilogram)
stickyAir.capacity = 100. * u.joule / (u.kelvin * u.kilogram)

Model.capacity    = 1000. * u.joule / (u.kelvin * u.kilogram)

In [9]:
# Density

#sediment.density          = GEO.LinearDensity(reference_density=2200. * u.kilogram / u.metre**3)
air.density               = 1. * u.kilogram / u.metre**3
stickyAir.density         = 1. * u.kilogram / u.metre**3

wz_lithosphere.density        = GEO.LinearDensity(2000. * u.kilogram / u.metre**3, thermalExpansivity=3e-5 / u.kelvin)
wz_basalticrust.density        = GEO.LinearDensity(2000. * u.kilogram / u.metre**3, thermalExpansivity=3e-5 / u.kelvin)

basalticrust.density        = GEO.LinearDensity(3000. * u.kilogram / u.metre**3, thermalExpansivity=3e-5 / u.kelvin)
Slab_basalticrust.density        = GEO.LinearDensity(3000. * u.kilogram / u.metre**3, thermalExpansivity=3e-5 / u.kelvin)
#midcrust.density          = GEO.LinearDensity(2620. * u.kilogram / u.metre**3, thermalExpansivity=3e-5 / u.kelvin)
#lowercrust.density        = GEO.LinearDensity(2720. * u.kilogram / u.metre**3, thermalExpansivity=3e-5 / u.kelvin)
lithosphere.density = GEO.LinearDensity(3300. * u.kilogram / u.metre**3, thermalExpansivity=3e-5 / u.kelvin)
Slab_lithosphere.density = GEO.LinearDensity(3300. * u.kilogram / u.metre**3, thermalExpansivity=3e-5 / u.kelvin)

asthenosphere.density     = GEO.LinearDensity(3395. * u.kilogram / u.metre**3, thermalExpansivity=3e-5 / u.kelvin)
mantle.density            = GEO.LinearDensity(4000. * u.kilogram / u.metre**3, thermalExpansivity=3e-5 / u.kelvin)


In [10]:
# Radiogenic Heat Production

#sediment.radiogenicHeatProd   = 0.60 * u.microwatt / u.meter**3 # faccenda 2008: 1-5

wz_lithosphere.radiogenicHeatProd = 0.25 * u.microwatt / u.meter**3 # faccenda 2008: 1-5
wz_basalticrust.radiogenicHeatProd = 0.25 * u.microwatt / u.meter**3 # faccenda 2008: 1-5


basalticrust.radiogenicHeatProd = 0.25 * u.microwatt / u.meter**3 # faccenda 2008: 1-5
Slab_basalticrust.radiogenicHeatProd = 0.25 * u.microwatt / u.meter**3 # faccenda 2008: 1-5
#midcrust.radiogenicHeatProd   = 0.40 * u.microwatt / u.meter**3 # faccenda 2008: 1-5
#lowercrust.radiogenicHeatProd = 0.30 * u.microwatt / u.meter**3 # faccenda 2008: 0.25, ocenic crust:0,25
lithosphere.radiogenicHeatProd = 0.022 * u.microwatt / u.meter**3 # faccenda 2008: 0.022
Slab_lithosphere.radiogenicHeatProd = 0.022 * u.microwatt / u.meter**3 # faccenda 2008: 0.022

asthenosphere.radiogenicHeatProd = 0.02e-6 * u.microwatt / u.meter**3
mantle.radiogenicHeatProd = 0.02e-6 * u.microwatt / u.meter**3

### Viscous Rheologies

In [11]:
rh = GEO.ViscousCreepRegistry() 

viscosity = GEO.ViscousCreep()
viscosity.activationEnergy = 200. * u.kilojoule

air.viscosity                = 1e18 * u.pascal * u.second
stickyAir.viscosity          = 1e19 * u.pascal * u.second # or water

# sediment.viscosity           = rh.Kronenberg_et_al_1984 # rh.Gleason_and_Tullis_1995
# sediment.minViscosity        = 1e18 * u.pascal * u.second
# sediment.maxViscosity        = 5e23 * u.pascal * u.second


wz_lithosphere.viscosity    = 1.0 * rh.Goetze_et_al_1978
wz_lithosphere.minViscosity = 1e18 * u.pascal * u.second
wz_lithosphere.maxViscosity = 5e23 * u.pascal * u.second
wz_basalticrust.viscosity    = 1.0 * rh.Goetze_et_al_1978
wz_basalticrust.minViscosity = 1e18 * u.pascal * u.second
wz_basalticrust.maxViscosity = 5e23 * u.pascal * u.second



basalticrust.viscosity    = 1.0 * rh.Goetze_et_al_1978
basalticrust.minViscosity = 1e18 * u.pascal * u.second
basalticrust.maxViscosity = 5e23 * u.pascal * u.second
Slab_basalticrust.viscosity    = 1.0 * rh.Goetze_et_al_1978
Slab_basalticrust.minViscosity = 1e18 * u.pascal * u.second
Slab_basalticrust.maxViscosity = 5e23 * u.pascal * u.second



# midcrust.viscosity    = 1.0 * rh.Gleason_and_Tullis_1995
# midcrust.minViscosity = 1e18 * u.pascal * u.second
# midcrust.maxViscosity = 5e23 * u.pascal * u.second

# lowercrust.viscosity    = 1.0 * rh.Wang_et_al_2012
# lowercrust.minViscosity = 1e18 * u.pascal * u.second
# lowercrust.maxViscosity = 5e23 * u.pascal * u.second

lithosphere.viscosity    = 1.0 * rh.Brace_and_Kohlstedt_1980 # 5.0 * rh.Karato_and_Wu_1990
lithosphere.minViscosity = 1e18 * u.pascal * u.second
lithosphere.maxViscosity = 5e23 * u.pascal * u.second
Slab_lithosphere.viscosity    = 1.0 * rh.Brace_and_Kohlstedt_1980 # 5.0 * rh.Karato_and_Wu_1990
Slab_lithosphere.minViscosity = 1e18 * u.pascal * u.second
Slab_lithosphere.maxViscosity = 5e23 * u.pascal * u.second


asthenosphere.viscosity    = rh.Brace_and_Kohlstedt_1980 
asthenosphere.minViscosity = 1e18 * u.pascal * u.second
asthenosphere.maxViscosity = 5e23 * u.pascal * u.second

mantle.viscosity    = rh.Brace_and_Kohlstedt_1980 # 1.0 * rh.Karato_and_Wu_1990
mantle.minViscosity = 1e18 * u.pascal * u.second
mantle.maxViscosity = 5e23 * u.pascal * u.second

minimumviscosity   = GEO.ConstantViscosity(1e18 * u.pascal * u.second)
maximumviscosity   = GEO.ConstantViscosity(5e23 * u.pascal * u.second)

# rh.Gleason_and_Tullis_1995


## Plasticities

In [12]:
pl = GEO.PlasticityRegistry()

In [13]:
# sediment.plasticity     = pl.Rey_et_al_2010_UpperCrust
# sediment.plasticity.frictionCoefficient = 0.268
# sediment.plasticity.frictionAfterSoftening = 0.01
# sediment.plasticity.epsilon2 = 0.10
# sediment.stressLimiter  = 20 * u.megapascal


wz_basalticrust.plasticity     = pl.Rey_et_al_2010_Mantle
wz_basalticrust.stressLimiter  = 300 * u.megapascal
wz_lithosphere.plasticity     = pl.Rey_et_al_2010_Mantle
wz_lithosphere.stressLimiter  = 300 * u.megapascal


basalticrust.plasticity     = pl.Rey_et_al_2010_UpperCrust
basalticrust.plasticity.frictionCoefficient = 0.268
basalticrust.plasticity.frictionAfterSoftening = 0.03
basalticrust.plasticity.epsilon2 = 0.15
basalticrust.stressLimiter  = 100 * u.megapascal

Slab_basalticrust.plasticity     = pl.Rey_et_al_2010_UpperCrust
Slab_basalticrust.plasticity.frictionCoefficient = 0.268
Slab_basalticrust.plasticity.frictionAfterSoftening = 0.03
Slab_basalticrust.plasticity.epsilon2 = 0.15
Slab_basalticrust.stressLimiter  = 100 * u.megapascal


# midcrust.plasticity     = pl.Rey_et_al_2010_UpperCrust
# midcrust.plasticity.frictionCoefficient = 0.268
# midcrust.plasticity.frictionAfterSoftening = 0.07
# midcrust.stressLimiter  = 120 * u.megapascal

# lowercrust.plasticity     = pl.Rey_et_al_2010_LowerCrust
# lowercrust.stressLimiter  = 150 * u.megapascal

lithosphere.plasticity     = pl.Rey_et_al_2010_Mantle
lithosphere.stressLimiter  = 300 * u.megapascal

Slab_lithosphere.plasticity     = pl.Rey_et_al_2010_Mantle
Slab_lithosphere.stressLimiter  = 300 * u.megapascal

asthenosphere.plasticity     = pl.Rey_et_al_2010_Mantle
asthenosphere.stressLimiter  = 380 * u.megapascal # Rey and Muller 2010

mantle.plasticity             = pl.Rey_et_al_2010_Mantle

# or

# mantle.plasticity = GEO.DruckerPrager(
#    cohesion=20.*u.megapascal,
#    cohesionAfterSoftening=20.*u.megapascal,
#    frictionCoefficient=0.12,
#    frictionAfterSoftening=0.05, # Tutu et al., 2018
#    epsilon1=0.01, epsilon2=1.0)


#pl.Rey_et_al_2010_UpperCrust


In [16]:
#basalticrust.plasticity

## Add Temperature Boundary Conditions

Temperature is 293.15K at the top and 3570K at the bottom. Temperature is constant in the mantle and the air layers. 

In [18]:
Model.set_temperatureBCs(top=293.15 * u.degK, 
                         bottom=3570. * u.degK, 
                         indexSets=[(mantle.indices, 3570. * u.degK),
                                    (stickyAir.indices, 293.15 * u.degK ),
                                    (air.indices, 293.15 * u.degK )]) 

#materials=[(air, 293.15 * u.degK),(stickyAir, 293.15 * u.degK), (asthenosphere.indices, 1603.15 * u.degK), (mantle.indices, 3570 * u.degK)])  #

## Add Velocity Boundary Conditions

We push on the right side. The back and front wall are freeslip. We use a pseudo isostatic support condition at the bottom.

In [19]:
Model.set_velocityBCs(left=[0., None, None],
                      right=[0., None, None],
                      back=[None, 0., None],
                      front=[None, 0., None],
                      bottom=[None, None, 0.],
                      top=[None, None, 0.])

# Add Passive Tracers 

In [21]:
#help(Model.add_passive_tracers)

In [22]:
import numpy as np

xp = np.linspace(GEO.nd(Model.minCoord[0]), GEO.nd(Model.maxCoord[0]), 100)
yp = np.linspace(GEO.nd(Model.minCoord[1]), GEO.nd(Model.maxCoord[1]), 100)

xp, yp = np.meshgrid(xp, yp)
xp = xp.flatten()
yp = yp.flatten()
zp = np.zeros(xp.shape)

surface_tracers = Model.add_passive_tracers(name="Surface2", vertices=[xp, yp, zp])
moho_tracers = Model.add_passive_tracers(name="Moho", vertices=[xp, yp, zp+GEO.nd(asthenosphere.top)])
#moho_tracers = Model.add_passive_tracers(name="Moho", vertices=[x,y-40.*u.kilometer])
#moho_tracers.add_tracked_field(Model.pressureField, name="Pressure" , dataType="float", units=u.megapascal, count=1, overwrite=True)
#surface_tracers.add_tracked_field(Model.pressureField, name="Pressure" , dataType="float", units=u.megapascal, count=1, overwrite=True)

#print moho_tracers
#print surface_tracers

import numpy as np

x2 = np.linspace(Model.minCoord[0], Model.maxCoord[0], 1000) * u.kilometer
y2 = np.linspace(Model.minCoord[0], Model.maxCoord[0], 1000) * u.kilometer
x2, y2 = np.meshgrid(xp, yp)
x2 = x2.flatten()
y2 = y2.flatten()
z2 = -80. * u.kilometer

level_tracers = Model.add_passive_tracers(name="Level", vertices=[x2,y2,z2])
level_tracers2 = Model.level_tracers.add_tracked_field(Model.strainRateField, name="Lev_StrainRate", dataType="float", units=u.sec, count=1, overwrite=True)

# Add ellipses

x_c, y_c, z_c = GEO.circles_grid(radius=2.0*u.kilometer, 
                    minCoord=[0. * u.kilometer, 0. * u.kilometer, lowercrust.bottom], 
                    maxCoord=[10000. * u.kilometer, 2000. * u.kilometer, uppercrust.top])
circles_c = Model.add_passive_tracers(name="FSE_Crust", vertices=[x_c, y_c, z_c])

# Initial model

In [23]:
# import glucifer
# Fig = glucifer.Figure(figsize=(1200,400))
# Fig.Points(surface_tracers.swarm, pointSize=2.0)
# Fig.Points(moho_tracers.swarm, pointSize=2.0)
# #Fig.Points(circles_c.swarm, pointSize=1.5)
# Fig.Points(Model.swarm, Model.materialField, fn_size=2.0)
# Fig.show()


In [24]:
# import numpy as np
# x2 = np.linspace(Model.minCoord[0], Model.maxCoord[0], 3000) * u.kilometer
# y2 = -80. * u.kilometer
# level_tracers = Model.add_passive_tracers(name="Level", vertices=[x2,y2])
# level_tracers2 = Model.level_tracers.add_tracked_field(Model.strainRateField, name="Lev_StrainRate", dataType="float", units=u.sec, count=1, overwrite=True)



# Surface Processes / BADLANDS

In [25]:
# Model.surfaceProcesses = GEO.surfaceProcesses.Badlands(airIndex=[air.index], sedimentIndex=sediment.index,
#                                            XML="ressources/badlands.xml", resolution=1. * u.kilometer, 
#                                            checkpoint_interval=0.01 * u.megayears)


#Model.surfaceProcesses = GEO.surfaceProcesses.SedimentationThreshold(Model.swarm, Model.materialField, air=[air, stickyAir], sediment=[sediment], threshold=0.)

In [26]:
GEO.rcParams["solver"] = "mg" # This is the default so that line is technically not needed.

In [27]:
Model.init_model()

In [ ]:
#Fig = Model.plot.temperature(script=["rotate z 30", "rotate x -60"], figsize=(900,600))

In [ ]:
#Fig = Model.plot.pressureField(script=["rotate z 30", "rotate x -60"], figsize=(900,600))

In [ ]:
mohoT = Model.temperature.evaluate(moho_tracers.swarm)[:,0].mean()
#print("Moho Temperature = ", GEO.Dimensionalize(mohoT, u.degC)) 

In [ ]:
Model.run_for( 20* u.millionyear, checkpoint_interval=1 * u.megayear)
u.
